# Busca em Strings - Estrutura de Dados

## KMP - Knuth-Morris-Pratt

In [ ]:
pattern = "ABABCABCAB"
text = "ABABDABACDABABCABCABCABCABC"

In [ ]:
def build_lps_table(pattern):
    m = len(pattern)
    lps = [0] * m
    length = 0
    i = 1

    while i < m:
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1
    return lps


In [ ]:
def kmp_search(text, pattern):
    n = len(text)
    m = len(pattern)
    lps = build_lps_table(pattern)
    # print(lps)
    matches = []

    i = j = 0
    while i < n:
        if text[i] == pattern[j]:
            i += 1
            j += 1

        if j == m:
            matches.append(i - j)
            j = lps[j - 1]
        elif i < n and text[i] != pattern[j]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1

    return matches


In [ ]:
kmp_search(text, pattern)

[10]

## Rabin-Karp

In [ ]:
def rabin_karp(text, pattern):
    n, m = len(text), len(pattern)
    d = 256  # tamanho do alfabeto
    q = 101  # número primo

    pattern_hash = text_hash = 0
    h = pow(d, m-1) % q

    # Calcula hash inicial
    for i in range(m):
        pattern_hash = (d * pattern_hash + ord(pattern[i])) % q
        text_hash = (d * text_hash + ord(text[i])) % q

    matches = []
    for i in range(n - m + 1):
        if pattern_hash == text_hash:
            # Verificação caractere por caractere
            if text[i:i+m] == pattern:
                matches.append(i)

        if i < n - m:
            text_hash = (d * (text_hash - ord(text[i]) * h) + ord(text[i + m])) % q
            if text_hash < 0:
                text_hash += q

    return matches


In [ ]:
rabin_karp(text, pattern)

[10]

## Aho-Corasick

In [2]:
from collections import defaultdict, deque
from typing import List, Dict, Tuple, Optional, Set
import re


class AhoCorasick:

    def __init__(self,
                 patterns: List[str],
                 case_sensitive: bool = True,
                 only_whole_words: bool = False):
        """
        Inicializa o automato Aho-Corasick.

        Args:
            patterns: Lista de padrões para buscar
            case_sensitive: Se True, busca é sensível a maiúsculas/minúsculas
            only_whole_words: Se True, busca apenas palavras completas
        """
        self.case_sensitive = case_sensitive
        self.only_whole_words = only_whole_words

        # Validação e preprocessamento dos padrões
        self.patterns = self._validate_and_preprocess_patterns(patterns)

        if not self.patterns:
            raise ValueError("Nenhum padrão válido fornecido")

        # Estruturas do automato
        self.goto = {}  # Função de transição (usando dict para economia de memória)
        self.fail = {}  # Função de falha
        self.output = defaultdict(set)  # Função de saída

        # Construção do automato
        self._build_goto_function()
        self._build_failure_function()

        # Regex para validação de palavras completas
        if self.only_whole_words:
            self.word_boundary = re.compile(r'\b')

    def _validate_and_preprocess_patterns(self, patterns: List[str]) -> List[str]:
        """Valida e preprocessa os padrões de entrada."""
        if not patterns:
            return []

        processed = []
        seen = set()

        for pattern in patterns:
            if not isinstance(pattern, str):
                continue

            # Preprocessamento baseado nas configurações
            if not self.case_sensitive:
                pattern = pattern.lower()

            # Remove duplicatas
            if pattern not in seen:
                processed.append(pattern)
                seen.add(pattern)

        return processed

    def _build_goto_function(self):
        """Constrói a função goto (trie)."""
        # Estado inicial
        self.goto[0] = {}
        state_count = 1

        # Constrói a trie
        for pattern_idx, pattern in enumerate(self.patterns):
            current_state = 0

            for char in pattern:
                if char not in self.goto[current_state]:
                    self.goto[current_state][char] = state_count
                    self.goto[state_count] = {}
                    state_count += 1

                current_state = self.goto[current_state][char]

            # Marca o estado final
            self.output[current_state].add(pattern_idx)

    def _build_failure_function(self):
        """Constrói a função de falha usando BFS."""
        # Inicializa a função de falha
        self.fail[0] = 0

        # Fila para BFS
        queue = deque()

        # Estados de profundidade 1 têm falha para o estado 0
        for char, state in self.goto[0].items():
            self.fail[state] = 0
            queue.append(state)

        # Processa os demais estados
        while queue:
            current_state = queue.popleft()

            for char, next_state in self.goto[current_state].items():
                queue.append(next_state)

                # Encontra o estado de falha
                failure_state = self.fail[current_state]

                while failure_state != 0 and char not in self.goto[failure_state]:
                    failure_state = self.fail[failure_state]

                if char in self.goto[failure_state]:
                    failure_state = self.goto[failure_state][char]

                self.fail[next_state] = failure_state

                # Combina as saídas
                self.output[next_state].update(self.output[failure_state])

    def search(self, text: str) -> Dict[str, List[Tuple[int, int]]]:
        """
        Busca todos os padrões no texto.

        Args:
            text: Texto onde buscar

        Returns:
            Dicionário com padrão -> lista de (início, fim) das ocorrências
        """
        if not text:
            return {}

        # Preprocessa o texto
        search_text = text if self.case_sensitive else text.lower()

        results = defaultdict(list)
        current_state = 0

        for i, char in enumerate(search_text):
            # Encontra o próximo estado
            while current_state != 0 and char not in self.goto[current_state]:
                current_state = self.fail[current_state]

            if char in self.goto[current_state]:
                current_state = self.goto[current_state][char]

            # Verifica se há padrões que terminam neste estado
            if current_state in self.output:
                for pattern_idx in self.output[current_state]:
                    pattern = self.patterns[pattern_idx]
                    start_pos = i - len(pattern) + 1
                    end_pos = i + 1

                    # Verifica se é palavra completa (se necessário)
                    if self.only_whole_words:
                        if not self._is_whole_word(text, start_pos, end_pos):
                            continue

                    results[pattern].append((start_pos, end_pos))

        return dict(results)

    def _is_whole_word(self, text: str, start: int, end: int) -> bool:
        """Verifica se a ocorrência é uma palavra completa."""
        # Verifica o caractere anterior
        if start > 0 and text[start - 1].isalnum():
            return False

        # Verifica o caractere posterior
        if end < len(text) and text[end].isalnum():
            return False

        return True

    def search_with_context(self,
                          text: str,
                          context_length: int = 20) -> Dict[str, List[Dict]]:
        """
        Busca padrões retornando contexto adicional.

        Args:
            text: Texto onde buscar
            context_length: Número de caracteres de contexto antes/depois

        Returns:
            Dicionário com informações detalhadas das ocorrências
        """
        results = self.search(text)
        detailed_results = defaultdict(list)

        for pattern, positions in results.items():
            for start, end in positions:
                context_start = max(0, start - context_length)
                context_end = min(len(text), end + context_length)

                detailed_results[pattern].append({
                    'start': start,
                    'end': end,
                    'match': text[start:end],
                    'context': text[context_start:context_end],
                    'context_start': context_start,
                    'context_end': context_end
                })

        return dict(detailed_results)

    def count_occurrences(self, text: str) -> Dict[str, int]:
        """Conta o número de ocorrências de cada padrão."""
        results = self.search(text)
        return {pattern: len(positions) for pattern, positions in results.items()}

    def get_statistics(self) -> Dict:
        """Retorna estatísticas sobre o automato construído."""
        return {
            'num_patterns': len(self.patterns),
            'num_states': len(self.goto),
            'total_pattern_length': sum(len(p) for p in self.patterns),
            'patterns': self.patterns.copy(),
            'case_sensitive': self.case_sensitive,
            'only_whole_words': self.only_whole_words
        }


if __name__ == "__main__":
    # print("=== Demonstração do Aho-Corasick ===\n")

    # Exemplo 1: Busca básica
    patterns = ["he", "she", "his", "hers"]
    text = "ushers"

    ac = AhoCorasick(patterns, case_sensitive=False)
    results = ac.search(text)

    print("Exemplo 1 - Busca básica:")
    print(f"Texto: '{text}'")
    print(f"Padrões: {patterns}")
    print("Resultados:")
    for pattern, positions in results.items():
        for start, end in positions:
            print(f"  '{pattern}' encontrado em posição {start}-{end}")

    # Exemplo 2: Busca com contexto
    print("\nExemplo 2 - Busca com contexto:")
    long_text = "The quick brown fox jumps over the lazy dog. She sells seashells by the seashore."
    patterns2 = ["the", "she", "sells"]

    ac2 = AhoCorasick(patterns2, case_sensitive=False)
    detailed_results = ac2.search_with_context(long_text, context_length=10)

    print(f"Texto: '{long_text}'")
    print(f"Padrões: {patterns2}")
    for pattern, matches in detailed_results.items():
        print(f"\nPadrão '{pattern}':")
        for _match in matches:
            print(f"  Posição {_match['start']}-{_match['end']}: '{_match['match']}'")
            print(f"  Contexto: '{_match['context']}'")

    # Exemplo 3: Estatísticas
    print("\nEstatísticas do automato:")
    stats = ac2.get_statistics()
    for key, value in stats.items():
        print(f"  {key}: {value}")


Exemplo 1 - Busca básica:
Texto: 'ushers'
Padrões: ['he', 'she', 'his', 'hers']
Resultados:
  'he' encontrado em posição 2-4
  'she' encontrado em posição 1-4
  'hers' encontrado em posição 2-6

Exemplo 2 - Busca com contexto:
Texto: 'The quick brown fox jumps over the lazy dog. She sells seashells by the seashore.'
Padrões: ['the', 'she', 'sells']

Padrão 'the':
  Posição 0-3: 'The'
  Contexto: 'The quick bro'
  Posição 31-34: 'the'
  Contexto: 'umps over the lazy dog.'
  Posição 68-71: 'the'
  Contexto: 'shells by the seashore.'

Padrão 'she':
  Posição 45-48: 'She'
  Contexto: 'lazy dog. She sells sea'
  Posição 58-61: 'she'
  Contexto: ' sells seashells by the'

Padrão 'sells':
  Posição 49-54: 'sells'
  Contexto: ' dog. She sells seashells'

Estatísticas do automato:
  num_patterns: 3
  num_states: 11
  total_pattern_length: 11
  patterns: ['the', 'she', 'sells']
  case_sensitive: False
  only_whole_words: False


> Busca em Strings